In [32]:
from lark import Lark, Token, Transformer, Tree
from typing import Dict, Tuple, Sequence, List
from typing import NewType
from pprint import pprint
import re


Token = Dict[str, str]
Block = Dict[str, List[Token]]


def token(_type: str, value: str) -> Token:
    """Returns a token with <_type> and <value>"""
    
    return {
        'value': value,
        'type': _type,
    }


def block( _id: str, content: List[Token]) -> Block:
    """
    Returns a block with <_id> and <content>
    <content> is a list of tokens
    """
    
    return {
        'id': _id,
        'content': content,
    }


c = [token('TXT', 'hello'), token('REF', 'world')]
pprint(block('0', c))

{'content': [{'type': 'TXT', 'value': 'hello'},
             {'type': 'REF', 'value': 'world'}],
 'id': '0'}


In [112]:
text = '''
- Earth
    - also called the [[World]]
    - home to [[Human]]s
- [[Max S]]
    - is - a [[Human]]
    - likes [[Programming]]
- John
    - is a [[Alien]]
    - likes [[Programming]]
    -
- [[Alien]]
  - takes you to a leader
- [[Max S]] is not an [[Alien]]
'''
lines = re.split(r'(\s*\-.*\n)', text)
for line in lines:
    line = line.replace('\n', '')
    if line and len(line.replace(' ', '')) > 0:
        line_text = re.sub(r'(^\s*\-\s*)', '', line)
        indent = len(line) - len(line.lstrip())
        
        references = re.findall(r'(\[\[[^\[|\]]+\]\])+', line)
        references = [re.sub(r'(\[\[)|(\]\])', '', x) for x in references]

        print(' ' * indent, line_text)
        

 Earth
     also called the [[World]]
     home to [[Human]]s
 [[Max S]]
     is - a [[Human]]
     likes [[Programming]]
 John
     is a [[Alien]]
     likes [[Programming]]
     
 [[Alien]]
   takes you to a leader
 [[Max S]] is not an [[Alien]]
